In [200]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
import warnings
warnings.filterwarnings('ignore')

In [221]:
data = pd.read_csv(f'icici/orig/2023_credit.csv', header=6, thousands=',')
# data = data.iloc[1:-11]
data.head()

,Date,Sr.No.,Transaction Details,Reward Point Header,Intl.Amount,Amount(in Rs),BillingAmountSign
0,4375 XXXX XXXX 8000,NaN,NaN,NaN,NaN,NaN,NaN
1,15-Mar-23,58,CLEARTRIP BANGALORE IN,1461.0,0.0,73061.05,73061.05
2,17-Mar-23,59,INSTANT EMI OFFUS CONVERSION,0.0,0.0,0.00,0.00
3,17-Mar-23,60,Processing Fee 199 : 0%,0.0,0.0,199.00,199.00
4,17-Mar-23,61,IGST-CI@18%,0.0,0.0,35.82,35.82


In [222]:
def preprocess_credit_data(year):
    data = pd.read_csv(f'icici/orig/20{year}_credit.csv', header=6, thousands=',')
    data = data.iloc[1:-11]
    data = data.drop(['Sr.No.', 'Reward Point Header', 'Intl.Amount', 'BillingAmountSign'], axis=1)
    data.rename(columns={"Transaction Details": "Description",
                        "Amount(in Rs)": "Amount"
                        }, inplace=True)
    data['Date'] = pd.to_datetime(data['Date'], format='%d-%b-%y')
    data = data.sort_values(by=['Date'], ascending=True)
    data[['Amount']] = data[['Amount']].astype(float)
    data.insert(loc = 1,
          column = 'Type',
          value = 'Credit')
    data.insert(loc = len(data.columns),
          column = 'Balance',
          value = 0)
    return data

In [202]:
def preprocess_debit_data(year):
    data = pd.read_excel(f'icici/orig/20{year}.xls')
    data = data.iloc[11:-28, 1:]
    data.columns = data.iloc[0]
    data = data.iloc[1:]
    data = data.set_index(data.columns[0])
    data.rename(columns={"Transaction Remarks": "Description",
                        "Withdrawal Amount (INR )": "Amount",
                        "Deposit Amount (INR )": "Deposit",
                        "Balance (INR )": "Balance",
                        "Value Date": "Date"
                        }, inplace=True)
    data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
    data = data.sort_values(by=['Date'], ascending=True)
    data[['Amount', 'Deposit', 'Balance']] = data[['Amount', 'Deposit', 'Balance']].astype(float)
    data['Amount'] = data['Deposit'] - data['Amount']
    data = data.drop(['Cheque Number', 'Transaction Date', 'Deposit'], axis=1)
    data.insert(loc = 1,
          column = 'Type',
          value = 'Debit')
    return data

In [203]:
def write_to_excel(data, filename):
    writer = pd.ExcelWriter(f"icici/clean/{filename}.xlsx",
                        date_format='YYYY-MM-DD',
                        datetime_format='YYYY-MM-DD')

    data.to_excel(writer, 
                index=False,
                freeze_panes=(1,0),
                sheet_name='Sheet_1',
                engine='xlsxwriter')

    workbook  = writer.book
    worksheet = writer.sheets['Sheet_1']
    wrap_format = workbook.add_format({'text_wrap': True})
    worksheet.set_column('A:B', 12)
    worksheet.set_column('C:C', 30, wrap_format)
    worksheet.set_column('D:F', 12)
    worksheet.set_column('G:G', 30, wrap_format)
    writer.save()

In [204]:
YEARS = ['23']

def merge_all_data():
    data_list = []
    for year in YEARS:
        data = preprocess_debit_data(year)
        data_list.append(data)
    full_data = pd.concat(data_list)
    full_data = full_data.sort_values(by=['Date'], ascending=True)
    return full_data

In [205]:
debit_data = merge_all_data()
write_to_excel(debit_data, '2023_debit')

In [223]:
credit_data = preprocess_credit_data('23')
write_to_excel(credit_data, '2023_credit')

In [224]:
data = pd.concat([debit_data, credit_data])
data = data.sort_values(by=['Date'], ascending=True)

In [225]:
data.head(10)

,Date,Type,Description,Amount,Balance
1,2023-01-03,Debit,IRM/USD1183.99@84.46GST/OWREM/20230103222428 ...,101065.00,9719.09
2,2023-01-06,Debit,INF/INFT/000055956077/living expense/Self ...,-200000.00,209719.09
3,2023-01-11,Debit,UPI/301101509853/UPI/kkzahida@okhdfc/Federal B...,30000.00,179719.09
4,2023-01-20,Debit,UPI/338696573545/Oid20159678788@/paybil3066@pa...,459.00,179260.09
5,2023-01-23,Debit,MMT/IMPS/302323764431/SACHIN K S/SBIN0001161 ...,20000.00,159260.09
6,2023-01-31,Debit,UPI/303169899417/Chiichaa vandi/kkzahida@okhdf...,4000.00,155260.09
7,2023-02-14,Debit,UPI/304598648217/KSFE and eduloa/kkzahida@okhd...,23000.00,132260.09
8,2023-02-17,Debit,UPI/304887066580/Oid20387186114@/add-money@pay...,668.65,131591.44
9,2023-03-02,Debit,UPI/306115175775/KSFE and edu lo/kkzahida-1@ok...,23000.00,108591.44
10,2023-03-02,Debit,BIL/NEFT/000615213385/closesbiloan/SACHIN K S/...,18529.00,90062.44


In [226]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92 entries, 1 to 11
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         90 non-null     datetime64[ns]
 1   Type         92 non-null     object        
 2   Description  90 non-null     object        
 3   Amount       90 non-null     float64       
 4   Balance      92 non-null     float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 4.3+ KB


In [227]:
high_data = data[(abs(data['Amount']) > 9999)]
write_to_excel(high_data, '2023_high')

## Older code

In [211]:
# searchfor = ['zahida k k', 'salim k m', 'dummy name'], # kkzahida@oksbi, 
# kkzahida@okicic, kkzahida-1@oksb, kkzahida@okhdfc, kmsalimkara@oks, kmsalimkara-1@o, kmsalimkara@oka
# sarinksalim@oka
# searchfor = ['sarinksalim']
# parents_data = data[data['Description'].str.lower().str.contains('|'.join(searchfor))]
# parents_data = parents_data[parents_data['Withdrawal'] > 1999]
# parents_data = parents_data.drop(['Deposit'], axis=1)
# display(parents_data.head())
# print(len(parents_data))